In [1]:
import cv2
import numpy as np
from ultralytics import YOLO
from tiki.mini import TikiMini
from IPython.display import display
import ipywidgets as widgets
import time

##############################################
# 0. 전역 객체 준비
##############################################
model = YOLO("./best.pt")

tiki = TikiMini()
tiki.set_motor_mode(tiki.MOTOR_MODE_PWM)

object_names = ['box','car','enemy','hazmat','missile','mortar','tank']

##############################################
# 1. 카메라 초기화
##############################################
pipeline = (
    "nvarguscamerasrc ! video/x-raw(memory:NVMM), width=320, height=240, format=NV12, framerate=30/1 ! "
    "nvvidconv ! video/x-raw, format=BGRx ! videoconvert ! "
    "video/x-raw, format=BGR ! appsink"
)

cap = cv2.VideoCapture(pipeline, cv2.CAP_GSTREAMER)
if not cap.isOpened():
    raise RuntimeError("카메라 열기 실패")

##############################################
# 2. Jupyter용 출력창 준비
##############################################
video_widget = widgets.Image(format='jpeg', width=640, height=480)
display(video_widget)

def frame_to_bytes(frame):
    _, buf = cv2.imencode('.jpg', frame, [int(cv2.IMWRITE_JPEG_QUALITY), 60])
    return buf.tobytes()

##############################################
# 3. Bird-eye 변환 준비
##############################################
bird_width, bird_height = 320, 240
src_points = np.float32([
    [60, 120], [260, 120], [310, 230], [10, 230]
])
dst_points = np.float32([
    [0,0], [bird_width,0], [bird_width,bird_height], [0,bird_height]
])
M = cv2.getPerspectiveTransform(src_points, dst_points)

##############################################
# ============================================================
#   1) 슬라이딩 윈도우 + 시각화  (원본 구조 유지)
# ============================================================
def sliding_window_center_with_vis(binary_mask, num_windows=8, margin=20, min_pixels=40):
    h, w = binary_mask.shape[:2]
    window_height = h // num_windows

    ys, xs = np.where(binary_mask == 255)

    vis = cv2.cvtColor(binary_mask, cv2.COLOR_GRAY2BGR)

    if len(xs) == 0:
        return None, vis

    # 하단부 x 기준점
    bottom_mask = ys >= int(h * 0.75)
    if np.sum(bottom_mask) > 0:
        current_x = int(np.mean(xs[bottom_mask]))
    else:
        current_x = int(np.mean(xs))

    centers = []
    pts = []

    for i in range(num_windows):
        y_low = h - (i + 1) * window_height
        y_high = h - i * window_height

        if y_low < 0: 
            y_low = 0
        if y_high <= y_low:
            continue

        # 해당 구간 내 픽셀
        inds = (ys >= y_low) & (ys < y_high)
        win_xs = xs[inds]
        if len(win_xs) == 0:
            continue

        # 동적 윈도우 폭
        x_min = max(0, current_x - margin)
        x_max = min(w - 1, current_x + margin)

        lane_inds = (win_xs >= x_min) & (win_xs <= x_max)

        if np.sum(lane_inds) < min_pixels:
            continue

        current_x = int(np.mean(win_xs[lane_inds]))

        cy = (y_low + y_high) // 2
        pts.append((current_x, cy))
        centers.append(current_x)

        cv2.rectangle(vis, (x_min, y_low), (x_max, y_high), (0, 255, 0), 1)
        cv2.circle(vis, (current_x, cy), 3, (255, 0, 255), -1)

    if len(centers) == 0:
        return int(np.mean(xs)), vis

    pts_np = np.array(pts, np.int32)
    cv2.polylines(vis, [pts_np], False, (255, 0, 255), 2)

    return int(np.mean(centers)), vis

# ============================================================
#   2) 중앙 흰색 라인 검출 + 디버그 정보 반환
# ============================================================
def detect_lane_center(frame_bgr, white_threshold=180):
    h, w = frame_bgr.shape[:2]

    # ROI (하단부)
    roi_y1 = int(h*0.60)
    roi_y2 = h
    roi_x1 = int(w*0.10)
    roi_x2 = int(w*0.90)
    roi = frame_bgr[roi_y1:roi_y2, roi_x1:roi_x2]
    roi_h, roi_w = roi.shape[:2]

    # 1) HSV 필터링
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

    S_MAX = 60
    V_MIN = 180

    lower_white = np.array([0, 0, V_MIN], dtype=np.uint8)
    upper_white = np.array([180, S_MAX, 255], dtype=np.uint8)
    white_mask = cv2.inRange(hsv, lower_white, upper_white)

    # 2) 노란색 제거
    lower_yellow = np.array([15, 80, 80], dtype=np.uint8)
    upper_yellow = np.array([40, 255, 255], dtype=np.uint8)
    yellow_mask = cv2.inRange(hsv, lower_yellow, upper_yellow)

    white_mask = cv2.bitwise_and(white_mask, cv2.bitwise_not(yellow_mask))
    white_mask = cv2.GaussianBlur(white_mask, (5, 5), 0)

    ys, xs = np.where(white_mask == 255)

    white_mask_bgr = cv2.cvtColor(white_mask, cv2.COLOR_GRAY2BGR)

    if len(xs) == 0:  
        return None, False, {"roi": roi, "white": white_mask_bgr, "sliding": white_mask_bgr.copy()}

    # 3) 슬라이딩 윈도우
    center_x_rel, sliding_vis = sliding_window_center_with_vis(white_mask)

    if center_x_rel is None:
        center_x_rel = int(np.mean(xs))

    center_x_frame = center_x_rel

    # STOP 가로선 검출
    x_range = np.max(xs) - np.min(xs)
    y_range = np.max(ys) - np.min(ys)
    is_horizontal_bar = (x_range > roi_w * 0.6 and y_range < roi_h * 0.25)

    return center_x_frame, is_horizontal_bar, {
        "roi": roi,
        "white": white_mask_bgr,
        "sliding": sliding_vis
    }


GST_ARGUS: Creating output stream
CONSUMER: Waiting until producer is connected...
GST_ARGUS: Available Sensor modes :
GST_ARGUS: 3264 x 2464 FR = 21.000000 fps Duration = 47619048 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 3264 x 1848 FR = 28.000001 fps Duration = 35714284 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 1920 x 1080 FR = 29.999999 fps Duration = 33333334 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 1640 x 1232 FR = 29.999999 fps Duration = 33333334 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 1280 x 720 FR = 59.999999 fps Duration = 16666667 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 1280 x 720 FR = 120.000005 fps Duration = 8333333 ; Analog Gain range min 1.000000, max 10.625000; Exposur

[ WARN:0@4.497] global cap_gstreamer.cpp:1777 open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


Image(value=b'', format='jpeg', height='480', width='640')

In [ ]:
##############################################
# 주행 파라미터
##############################################
frame_center_x = 160
base_speed = 50
max_steering = 25

# PID
class PID:
    def __init__(self, kp=0.6, ki=0.0, kd=0.15):
        self.kp = kp
        self.ki = ki
        self.kd = kd
        self.integral = 0
        self.prev_error = 0

    def compute(self, error):
        self.integral += error
        derivative = error - self.prev_error
        self.prev_error = error
        self.integral = np.clip(self.integral, -200, 200)
        return self.kp * error + self.ki * self.integral + self.kd * derivative

pid = PID()

##############################################
# 메인 루프
##############################################
try:
    while True:
        ret, frame = cap.read()
        if not ret:
            continue

        frame = cv2.flip(frame, -1)

        # 복제
        original = frame.copy()
        bird = cv2.warpPerspective(original, M, (bird_width, bird_height))

        # Lane detection
        center_x, is_horizontal_bar, dbg = detect_lane_center(bird)

        if center_x is not None:
            error = center_x - frame_center_x

            steering = pid.compute(error)
            steering *= 0.5
            steering = float(np.clip(steering, -max_steering, max_steering))
            if is_horizontal_bar:
                steering = 0

            L = int(np.clip(base_speed + steering, 0, 127))
            Rm = int(np.clip(base_speed - steering, 0, 127))

            tiki.set_motor_power(tiki.MOTOR_LEFT, L)
            tiki.set_motor_power(tiki.MOTOR_RIGHT, Rm)

        else:
            tiki.stop()

        panel = cv2.hconcat([
            cv2.resize(original, (320,240)),
            cv2.resize(bird, (320,240))
        ])

        video_widget.value = frame_to_bytes(panel)
        time.sleep(0.02)

except KeyboardInterrupt:
    tiki.stop()
    print("STOPPED")

In [2]:
# ============================
# 셀 3 - 디버그 뷰 전용 루프
# YOLO + Bird-eye + 마스크 + 슬라이딩 윈도우 2x2 패널
# 모터 제어 없음 (카메라 확인용)
# ============================
import time
import cv2
import numpy as np

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            continue

        # 카메라 방향 보정
        frame = cv2.flip(frame, -1)
        original = frame.copy()

        # Bird-eye 변환
        bird = cv2.warpPerspective(original, M, (bird_width, bird_height))

        # Lane 검출 + 디버그 이미지
        center_x, is_horizontal_bar, dbg = detect_lane_center(bird)

        roi_dbg    = dbg["roi"]
        white_dbg  = dbg["white"]
        sliding_dbg = dbg["sliding"]

        # 크기 통일 (320x240)
        original_show = cv2.resize(original, (320, 240))
        bird_show     = cv2.resize(bird, (320, 240))
        white_show    = cv2.resize(white_dbg, (320, 240))
        sliding_show  = cv2.resize(sliding_dbg, (320, 240))

        # -----------------------
        # YOLO 검출 + 시각화
        # -----------------------
        results = model(original_show, verbose=False)[0]

        if hasattr(results, "boxes") and results.boxes is not None:
            boxes  = results.boxes
            xyxy   = boxes.xyxy.cpu().numpy()
            cls_ids = boxes.cls.cpu().numpy().astype(int)

            for box, cls_id in zip(xyxy, cls_ids):
                x1, y1, x2, y2 = box.astype(int)
                if 0 <= cls_id < len(object_names):
                    label = object_names[cls_id]
                else:
                    label = str(cls_id)

                cv2.rectangle(original_show, (x1, y1), (x2, y2), (0, 255, 0), 1)
                cv2.putText(
                    original_show, label,
                    (x1, max(0, y1 - 5)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.4,
                    (0, 255, 0), 1, cv2.LINE_AA
                )

        # -----------------------
        # 2x2 패널 구성
        # -----------------------
        top_row    = cv2.hconcat([original_show, bird_show])    # 좌: 원본+YOLO, 우: Bird-eye
        bottom_row = cv2.hconcat([white_show, sliding_show])    # 좌: 마스크, 우: 슬라이딩 윈도우
        panel      = cv2.vconcat([top_row, bottom_row])         # (640x480)

        # 위젯에 표시
        video_widget.value = frame_to_bytes(panel)

        time.sleep(0.02)

except KeyboardInterrupt:
    print("DEBUG VIEW STOPPED")


DEBUG VIEW STOPPED
